In [2]:
import pandas as pd
import numpy as np
from pandas import DatetimeIndex
import datetime, time

In [ ]:
df = pd.read_csv('D:\\trip_number_automation.csv')

#df['Odometer'] = df['Odometer'].apply(lambda x: x.round(decimals = 0))

In [7]:
from time import gmtime, strftime

curr_time_gmt = strftime('%Y-%m-%d %H:%M:%S', gmtime())

print curr_time_gmt

2016-11-04 09:47:10


In [12]:
print (datetime.datetime.utcnow() - datetime.timedelta(1)).isoformat()
# print pd.datetime(dt).isoformat()

2016-12-18T12:25:52.082000


In [22]:
day = (datetime.datetime.now() - datetime.timedelta(1)).day
month = (datetime.datetime.now() - datetime.timedelta(1)).month
year = (datetime.datetime.now()- datetime.timedelta(1)).year

print day
print month
print year

# pd.to_datetime(pd.datetime(year, month, day, 18, 30, 0, 0).isoformat())

t = pd.datetime(year, month, day)
t1 = ObjectId.from_datetime(t)
print t1




10
11
2016


NameError: name 'ObjectId' is not defined

In [16]:
(datetime.datetime.now() - datetime.timedelta(1)).time()


datetime.time(15, 26, 32, 589000)

In [10]:
(datetime.datetime.utcnow() - datetime.timedelta(1)).isoformat()

'2017-05-14T09:54:55.643000'

In [3]:
'ISODate('+ datetime.datetime.utcnow().isoformat()+')'

'ISODate(2017-05-15T09:52:55.874000)'

In [148]:
import datetime
import dateutil.parser

def getDateTimeFromISO8601String(s):
    d = dateutil.parser.parse(s)
    print d
    print type(d)


getDateTimeFromISO8601String('2016-11-08T09:24:31.600000')




2016-11-08 09:24:31.600000
<type 'datetime.datetime'>


In [11]:
import os
from django.utils import timezone
from datetime import datetime
import pytz

In [13]:
dt = datetime.datetime('2016-11-08', timezone.utc)
print dt

NameError: name 'timezone' is not defined

In [91]:
grouped = df.groupby(['assetName','date_use','trip'])

# pd.DataFrame(g.size().reset_index(name = 'group')).group[0] - pd.DataFrame(g.size().reset_index(name = 'group')).group[1]



In [111]:
for name, group in grouped:
    
    g = pd.DataFrame(group)
    
    
    print g['Odometer'].iloc[0]
    
    
    print g['assetName'].iloc[0], g['date_use'].iloc[0], g['trip'].iloc[0], g['Odometer'].iloc[0], g['Odometer'].iloc[0] - g['Odometer'].iloc[-1]
    print g['assetName'].iloc[-1], g['date_use'].iloc[-1], g['trip'].iloc[-1], g['Odometer'].iloc[-1]
    
#     for index, row in group.iterrows():
        
        
    

17778.3
Hyundai Creta 2016-10-25 1.0 17778.3 -51.0
Hyundai Creta 2016-10-25 1.0 17829.3
17829.3
Hyundai Creta 2016-10-25 2.0 17829.3 -6.0
Hyundai Creta 2016-10-25 2.0 17835.3
17835.3
Hyundai Creta 2016-10-25 3.0 17835.3 -4.0
Hyundai Creta 2016-10-25 3.0 17839.3
17840.3
Hyundai Creta 2016-10-25 4.0 17840.3 -57.0
Hyundai Creta 2016-10-25 4.0 17897.3
20544.47
Polo GT 2016-10-25 1.0 20544.47 -3.25
Polo GT 2016-10-25 1.0 20547.72
20547.83
Polo GT 2016-10-25 2.0 20547.83 -13.44
Polo GT 2016-10-25 2.0 20561.27
20561.27
Polo GT 2016-10-25 3.0 20561.27 -14.41
Polo GT 2016-10-25 3.0 20575.68


In [ ]:
# remove telematics data points sent when vehicle was idle or switched off

df['dup_loc'] = df.groupby(['customData_date','lat', 'lon', 'speed','Odometer']).cumcount().apply(lambda x: x + 1)
df_base = df.loc[(df['assetName'] == 'Polo GT') & (df['dup_loc'] == 1), :].copy()
df = df_base.reset_index(drop = True)

In [111]:
df_base['date_utc'] = pd.to_datetime(df_base['date']).dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata').dt.tz_localize(None)

In [129]:
df_base['date_use'] = df_base['date_utc'].apply(lambda x : x.strftime('%Y-%m-%d'))

In [130]:
df_base.date_use.head()

0    2016-09-19
1    2016-09-19
2    2016-09-19
3    2016-09-19
4    2016-09-19
Name: date_use, dtype: object

In [112]:
group_by_date = df_base.groupby('customData_date')

for name in group_by_date:
    df_base['time_diff'] = (df_base['date_utc'] - df_base['date_utc'].shift(1)).apply(lambda x: x/np.timedelta64(1, 'm'))

In [113]:
df_base['row_num']  = df_base.groupby('customData_date').cumcount().apply(lambda x : x + 1)

for name, group in df_base.groupby(['customData_date']):
    for index, row in df_base.iterrows():
        if df_base.loc[index, 'row_num'] == 1:
            df_base.loc[index, 'trip'] = 1
        else:
            if df_base.loc[index, 'time_diff'] > 20.0:
                df_base.loc[index, 'trip'] = df_base.loc[index-1, 'trip'] + 1
            else:
                df_base.loc[index,'trip'] = df_base.loc[index-1, 'trip']
                

In [114]:
df_base[['customData_date','date_utc', 'time_diff', 'row_num', 'trip']].to_csv('D:\\temp.csv', index = False)

In [12]:
crime_df = pd.read_csv('https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD', nrows = 50000)
crime_df.to_csv('D:\\crime_mongo_sample.csv', index = False)